In [24]:
 # Import Modules 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import kerastuner as kt
from mmdata import * 


In [25]:
# Import the final data csv
data = pd.read_csv('final_data.csv')
data 

,YEAR,TEAM,TEAM NO,SEED,ROUND,POWER RATING,POWER RATING RANK,NET RPI,RESUME,WAB RANK,...,POWER-PATH,SEED WON,SEED LOST,SEED DIFF,FIRST ROUND,SECOND ROUND,SWEET 16,ELITE 8,FINAL 4,TOTAL
0,2016,Arizona,603,6,64,89.0,59,26,27,23,...,4.4,119,51,68,10,2,0,1,0,13
1,2016,Austin Peay,602,16,64,68.8,462,189,218,229,...,-41.8,119,51,68,10,2,0,1,0,13
2,2016,Baylor,601,5,64,85.5,152,25,25,22,...,5.1,119,51,68,10,2,0,1,0,13
3,2016,Buffalo,600,14,64,75.7,396,91,112,129,...,-24.4,119,51,68,10,2,0,1,0,13
4,2016,Butler,599,9,32,84.2,194,56,29,32,...,-3.5,119,51,68,10,2,0,1,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,2021,Oregon St.,830,12,8,79.7,332,91,32,47,...,-19.5,11,1,10,9,6,2,1,0,18
780,2021,USC,817,6,8,85.8,143,19,37,7,...,7.5,11,1,10,9,6,2,1,0,18
781,2021,Arkansas,872,3,8,86.3,129,14,20,12,...,7.6,11,1,10,9,6,2,1,0,18
782,2021,UCLA,819,11,4,81.9,267,46,55,24,...,-6.8,11,1,10,9,6,2,1,0,18


In [26]:
data.columns

Index(['YEAR', 'TEAM', 'TEAM NO', 'SEED', 'ROUND', 'POWER RATING',
       'POWER RATING RANK', 'NET RPI', 'RESUME', 'WAB RANK', 'ELO', 'B POWER',
       'Q1 W', 'Q2 W', 'Q1 PLUS Q2 W', 'Q3 Q4 L', 'PLUS 500', 'R SCORE',
       'BY YEAR NO', 'BY ROUND NO', 'CURRENT ROUND', 'SCORE', 'PAKE',
       'PAKE RANK', 'PASE', 'PASE RANK', 'GAMES', 'W', 'L', 'WIN%', 'R64',
       'R32', 'S16', 'E8', 'F4', 'F2', 'CHAMP', 'TOP2', 'F4%', 'CHAMP%',
       'POWER', 'PATH', 'DRAW', 'WINS', 'POOL VALUE', 'POOL S-RANK',
       'NCAA S-RANK', 'VAL Z-SCORE', 'POWER-PATH', 'SEED WON', 'SEED LOST',
       'SEED DIFF', 'FIRST ROUND', 'SECOND ROUND', 'SWEET 16', 'ELITE 8',
       'FINAL 4', 'TOTAL'],
      dtype='object')

In [27]:
# List features and target
X = data.drop(columns='CHAMP')
y = data['CHAMP']
input_nodes = X.shape[1]
print(X.dtypes, y.dtypes)

YEAR                   int64
TEAM                  object
TEAM NO                int64
SEED                   int64
ROUND                  int64
POWER RATING         float64
POWER RATING RANK      int64
NET RPI                int64
RESUME                 int64
WAB RANK               int64
ELO                    int64
B POWER              float64
Q1 W                   int64
Q2 W                   int64
Q1 PLUS Q2 W           int64
Q3 Q4 L                int64
PLUS 500               int64
R SCORE              float64
BY YEAR NO             int64
BY ROUND NO            int64
CURRENT ROUND          int64
SCORE                float64
PAKE                 float64
PAKE RANK              int64
PASE                 float64
PASE RANK              int64
GAMES                  int64
W                      int64
L                      int64
WIN%                 float64
R64                    int64
R32                    int64
S16                    int64
E8                     int64
F4            

In [28]:
# Encode the data
X_encoder = OneHotEncoder()
y_encoder = LabelEncoder()

X = X_encoder.fit_transform(X)
y = y_encoder.fit_transform(y)

In [29]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# Intialize and fit robust scaler... best for this data
scaler = RobustScaler(with_centering=False)

X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [31]:
# Method for creating a model with hyperparameters
def create_model(hp):
    model = tf.keras.models.Sequential()
    activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
    # Decide number of neurons in each layer
    model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=32,
        max_value=512,
        step=32), activation=activation, input_dim=X.shape[1]))
    # Decide number of hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(tf.keras.layers.Dense(units=hp.Int(
            f'layer_{i}_units',
            min_value=32,
            max_value=512,
            step=32), activation=activation))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [32]:
# Initialize the tuner
tuner = kt.Hyperband(create_model,
                        objective='val_accuracy',
                        max_epochs=100,
                        hyperband_iterations=2)

Reloading Tuner from .\untitled_project\tuner0.json


In [33]:
# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

In [34]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

{'activation': 'tanh',
 'first_units': 448,
 'num_layers': 5,
 'layer_0_units': 416,
 'layer_1_units': 320,
 'layer_2_units': 32,
 'layer_3_units': 320,
 'layer_4_units': 384,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 4,
 'tuner/round': 0}

In [35]:
# Fit the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)
model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))



Epoch 1/100
20/20 [==============================] - 1s 13ms/step - loss: -1.1528 - accuracy: 0.8517 - val_loss: -3.3775 - val_accuracy: 0.8662
Epoch 2/100
20/20 [==============================] - 0s 9ms/step - loss: -3.1905 - accuracy: 0.9043 - val_loss: -4.4874 - val_accuracy: 0.8790
Epoch 3/100
20/20 [==============================] - 0s 10ms/step - loss: -4.0569 - accuracy: 0.9043 - val_loss: -5.4565 - val_accuracy: 0.8790
Epoch 4/100
20/20 [==============================] - 0s 10ms/step - loss: -4.8055 - accuracy: 0.9043 - val_loss: -6.4751 - val_accuracy: 0.8790
Epoch 5/100
20/20 [==============================] - 0s 10ms/step - loss: -5.5850 - accuracy: 0.9043 - val_loss: -7.3932 - val_accuracy: 0.8790
Epoch 6/100
20/20 [==============================] - 0s 9ms/step - loss: -6.3232 - accuracy: 0.9043 - val_loss: -8.3202 - val_accuracy: 0.8790
Epoch 7/100
20/20 [==============================] - 0s 10ms/step - loss: -7.0698 - accuracy: 0.9043 - val_loss: -9.1990 - val_accuracy: 0

In [36]:
# Evaluate the model
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5/5 - 0s - loss: -8.8933e+01 - accuracy: 0.8790 - 14ms/epoch - 3ms/step
Loss: -88.93331909179688, Accuracy: 0.8789808750152588
